Experimento 16 - Linear Interpolation Imputation, MAD outlier detection, Z-Score normalization, MLP classifier

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn.objects as so
from ucimlrepo import fetch_ucirepo

In [2]:
beans = fetch_ucirepo(id=602)
df = beans.data.features
targets = beans.data.targets

In [3]:
cols = ['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength', 'AspectRatio', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent', 'Solidity', 'Roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2', 'ShapeFactor3', 'ShapeFactor4']

In [4]:
import random

#Introducing Missing values (5%)

for index, i in enumerate(df):
  for jndex, j in enumerate(df[i]):
    if random.randint(0,100) < 5:
      df.loc[jndex,i] = np.NaN

<ipython-input-4-fbb7389abead>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i][jndex] = np.NaN


In [5]:
df.isna().sum()

Area               676
Perimeter          648
MajorAxisLength    671
MinorAxisLength    732
AspectRatio        652
Eccentricity       678
ConvexArea         668
EquivDiameter      703
Extent             660
Solidity           653
Roundness          641
Compactness        691
ShapeFactor1       670
ShapeFactor2       678
ShapeFactor3       697
ShapeFactor4       647
dtype: int64

In [6]:
df

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,NaN,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,182.734419,1.097356,NaN,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,NaN
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,NaN,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,NaN,231.526798,NaN,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,NaN,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,NaN,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [7]:
#Imputing Missing Values with Linear Interpolation
df_imputed = df.interpolate(method="nearest", order=3, limit=None,
                            limit_direction='both').ffill().bfill()
df_imputed

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,28395.0,610.291,200.524796,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724
1,28734.0,638.018,200.524796,182.734419,1.097356,0.549812,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430
2,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.998430
3,30008.0,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199
4,30140.0,620.134,201.847882,190.279279,1.060798,0.333680,30724.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13606,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385
13607,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42508.0,231.526798,0.714574,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219
13608,42101.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.705389,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767
13609,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222


In [8]:
#adding the labels
df_imputed['Class'] = targets
df_pre_missing_values = df.copy()
print(df_imputed.groupby('Class').agg(['count']))
df_imputed['Class'] = df_imputed['Class'].transform(lambda x: 0 if x == 'BARBUNYA' else (1 if x == 'BOMBAY' else (2 if x == 'CALI' else (3 if x == 'DERMASON' else (4 if x == 'HOROZ' else (5 if x == 'SEKER' else 6))))))

          Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count     count           count           count       count   
Class                                                                  
BARBUNYA  1322      1322            1322            1322        1322   
BOMBAY     522       522             522             522         522   
CALI      1630      1630            1630            1630        1630   
DERMASON  3546      3546            3546            3546        3546   
HOROZ     1928      1928            1928            1928        1928   
SEKER     2027      2027            2027            2027        2027   
SIRA      2636      2636            2636            2636        2636   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                                      
BARBUNYA         1322       1322          1322   13

In [9]:
#Outlier Removal with MAD
from scipy import stats

print(f'Pre Outlier Shape: {df_imputed.shape}')

df_no_outliers = df_imputed.copy()
print(df_no_outliers['Class'].unique())
for i in df_no_outliers['Class'].unique():
    class_unique = df_no_outliers[df_no_outliers['Class'] == i]
    for feature in class_unique:
      mad = 1.4826 * np.median(np.absolute(class_unique[feature] - class_unique[feature].median()))
      #print(mad)
      upper = class_unique[feature].median() + (3 * mad)
      lower = class_unique[feature].median() - (3 * mad)
      excluded_lower = pd.Series(class_unique[class_unique[feature] < lower].index)
      excluded_upper = pd.Series(class_unique[class_unique[feature] > upper].index)
      df_no_outliers.drop(excluded_lower.values, inplace = True, errors='ignore')
      df_no_outliers.drop(excluded_upper.values, inplace = True, errors='ignore')


print(f'Pos Outlier Shape: {df_no_outliers.shape}')

Pre Outlier Shape: (13611, 17)
[5 0 1 2 4 6 3]
Pos Outlier Shape: (12130, 17)


In [10]:
df_no_outliers['Class'] = df_no_outliers['Class'].transform(lambda x: 'BARBUNYA' if x == 0 else ('BOMBAY' if x == 1 else ('CALI' if x == 2 else ('DERMASON' if x == 3 else ('HOROZ' if x == 4 else ('SEKER' if x == 5 else 'SIRA'))))))
df_no_outliers = df_no_outliers.reset_index()
print(df_no_outliers.groupby('Class').agg(['count']))

         index  Area Perimeter MajorAxisLength MinorAxisLength AspectRatio  \
         count count     count           count           count       count   
Class                                                                        
BARBUNYA  1186  1186      1186            1186            1186        1186   
BOMBAY     472   472       472             472             472         472   
CALI      1510  1510      1510            1510            1510        1510   
DERMASON  3190  3190      3190            3190            3190        3190   
HOROZ     1623  1623      1623            1623            1623        1623   
SEKER     1752  1752      1752            1752            1752        1752   
SIRA      2397  2397      2397            2397            2397        2397   

         Eccentricity ConvexArea EquivDiameter Extent Solidity Roundness  \
                count      count         count  count    count     count   
Class                                                              

In [11]:
df_no_outliers = df_no_outliers.drop(['index'], axis='columns')
df_no_outliers

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395.0,610.291,200.524796,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,29380.0,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.998430,SEKER
2,30279.0,634.927,212.560556,190.279279,1.171067,0.520401,30600.0,196.347702,0.775688,0.989510,0.943852,0.970516,0.006697,0.003153,0.853270,0.999236,SEKER
3,30477.0,629.727,212.996755,182.737204,1.165591,0.513760,30847.0,197.124320,0.770682,0.989367,0.967109,0.925480,0.006979,0.003158,0.856514,0.998345,SEKER
4,30685.0,635.681,213.534145,183.157146,1.165852,0.514081,31044.0,197.659696,0.771561,0.988436,0.954240,0.925658,0.006959,0.003152,0.856844,0.998953,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12125,42097.0,759.696,288.721612,185.944705,1.552728,0.765002,42508.0,231.515799,0.714574,0.990331,0.916603,0.801865,0.006858,0.001749,0.642988,0.998385,DERMASON
12126,42101.0,757.499,281.576392,190.713136,1.476439,0.735702,42508.0,231.526798,0.714574,0.990752,0.922015,0.822252,0.006688,0.001886,0.676099,0.998219,DERMASON
12127,42101.0,759.321,281.539928,191.187979,1.472582,0.734065,42569.0,231.631261,0.705389,0.989899,0.918424,0.822730,0.006681,0.001888,0.676884,0.996767,DERMASON
12128,42147.0,763.779,283.382636,190.275731,1.489326,0.741055,42667.0,231.653247,0.705389,0.987813,0.907906,0.817457,0.006724,0.001852,0.668237,0.995222,DERMASON


In [12]:
#Normalization with Z-Score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(df_no_outliers[cols])
df_scaled = pd.DataFrame(scaler.transform(df_no_outliers[cols]), columns = cols)

df_scaled['Class'] = df_no_outliers['Class']
df_scaled

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRatio,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,Roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,-0.849424,-1.145944,-1.405212,-0.640209,-1.576525,-2.250366,-0.849771,-1.073464,0.266597,0.308275,1.429683,1.863274,0.703732,2.429034,1.952619,0.959286,SEKER
1,-0.815727,-1.081284,-1.261370,-0.594555,-1.525398,-2.106387,-0.816886,-1.018095,0.556603,0.510566,1.250931,1.787991,0.625112,2.260554,1.867142,0.869706,SEKER
2,-0.784971,-1.030671,-1.264475,-0.273831,-1.683197,-2.578249,-0.786194,-0.968365,0.507050,0.496470,1.180719,2.802078,0.134245,2.438324,2.147558,1.115490,SEKER
3,-0.778198,-1.055002,-1.259375,-0.442419,-1.705558,-2.652285,-0.777863,-0.955214,0.404732,0.455328,1.589194,2.062386,0.387479,2.447642,2.180762,0.843533,SEKER
4,-0.771082,-1.027143,-1.253091,-0.433032,-1.704490,-2.648703,-0.771218,-0.946149,0.422709,0.187313,1.363166,2.065311,0.369311,2.436245,2.184137,1.029193,SEKER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12125,-0.380671,-0.446868,-0.373908,-0.370722,-0.124789,0.148596,-0.384559,-0.372851,-0.741898,0.732914,0.702146,0.032033,0.279174,0.062857,-0.004579,0.855949,DERMASON
12126,-0.380534,-0.457148,-0.457458,-0.264133,-0.436293,-0.178043,-0.384559,-0.372665,-0.741898,0.853926,0.797202,0.366885,0.126262,0.294265,0.334296,0.805113,DERMASON
12127,-0.380534,-0.448622,-0.457885,-0.253518,-0.452045,-0.196297,-0.382501,-0.370896,-0.929608,0.608427,0.734128,0.374729,0.120072,0.298388,0.342335,0.362221,DERMASON
12128,-0.378960,-0.427763,-0.436338,-0.273910,-0.383674,-0.118372,-0.379196,-0.370524,-0.929608,0.007935,0.549405,0.288133,0.158176,0.237048,0.253832,-0.109189,DERMASON


In [13]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate


x_train = df_scaled.iloc[:, 0:16]
y_train = df_scaled.iloc[:, 16]
classifier = MLPClassifier(activation='logistic', solver='adam', alpha=1e-5, hidden_layer_sizes=(12, 3), random_state=1, verbose=True, learning_rate_init=0.3, tol=1e-3, max_iter=500)
scoring = {'acc' : 'accuracy',
           'prec' : 'precision_macro',
           'recall' : 'recall_macro',
           'f1' : 'f1_macro'}


y_pred = cross_validate(classifier, x_train, y_train, cv=10, scoring=scoring, return_train_score=True)
print('Experimento 16')
print('Acurácia Média: ' + '%.2f' % (np.mean(y_pred['test_acc'])*100) + '%')
print('Precisão Média: ' + '%.2f' % (np.mean(y_pred['test_prec'])*100) + '%')
print('Revocação Média: ' + '%.2f' % (np.mean(y_pred['test_recall'])*100) + '%')
print('F1-Score Médio: ' + '%.2f' % (np.mean(y_pred['test_f1'])*100) + '%')


Iteration 1, loss = 0.95555118
Iteration 2, loss = 0.47396051
Iteration 3, loss = 0.41452530
Iteration 4, loss = 0.37504129
Iteration 5, loss = 0.33157933
Iteration 6, loss = 0.30402413
Iteration 7, loss = 0.27859203
Iteration 8, loss = 0.21245152
Iteration 9, loss = 0.20023929
Iteration 10, loss = 0.17576242
Iteration 11, loss = 0.17534171
Iteration 12, loss = 0.18311234
Iteration 13, loss = 0.21587339
Iteration 14, loss = 0.17415131
Iteration 15, loss = 0.18376721
Iteration 16, loss = 0.17500352
Iteration 17, loss = 0.16245746
Iteration 18, loss = 0.16490966
Iteration 19, loss = 0.18269937
Iteration 20, loss = 0.15936810
Iteration 21, loss = 0.16348540
Iteration 22, loss = 0.16230422
Iteration 23, loss = 0.15643810
Iteration 24, loss = 0.19326494
Iteration 25, loss = 0.19930598
Iteration 26, loss = 0.18337253
Iteration 27, loss = 0.17190410
Iteration 28, loss = 0.15437301
Iteration 29, loss = 0.16050174
Iteration 30, loss = 0.15431637
Iteration 31, loss = 0.15813886
Iteration 32, los